In [38]:
import csv
import codecs
from tqdm import tqdm
from urllib.request import urlretrieve

In [39]:
rows = []
with codecs.open('../data/HomologyTAPE/pdb_chain_scop_uniprot.tsv', encoding='utf-8') as f:
    for row in csv.DictReader(f, skipinitialspace=True, delimiter='\t'):
        rows.append(row)
print(len(rows))
print(rows[0])

104840
{'PDB': '101m', 'CHAIN': 'A', 'SP_PRIMARY': 'P02185', 'SUNID': '15125', 'SCOP_ID': 'd101ma_'}


In [40]:

fold_dic = {}
uniprot_dic = {}
for row in rows:
    fold_dic[row['SCOP_ID']] = {}
    fold_dic[row['SCOP_ID']]['PDB-chain'] = row['PDB'].upper() + '-' + row['CHAIN']
    fold_dic[row['SCOP_ID']]['uniprot'] = row['SP_PRIMARY']
    pdb_chain = row['PDB'].upper() + '-' + row['CHAIN']
    if pdb_chain not in uniprot_dic.keys():
        uniprot_dic[pdb_chain] = []
    uniprot_dic[pdb_chain].append(row['SP_PRIMARY'])

In [41]:
multi_uniprot = []
for key, value in uniprot_dic.items():
    uniprot_dic[key] = list(set(value))
    if len(uniprot_dic[key]) > 1:
        multi_uniprot.append(key)

In [42]:
fold_dic['d101ma_']

{'PDB-chain': '101M-A', 'uniprot': 'P02185'}

In [43]:
fold_pdb = {}

In [44]:

with open('../data/HomologyTAPE/test_fold.txt') as f:
    test_file_lines = f.readlines()
test_lines = {}
test_pdb = {}
failed_test_scop = []
for line in test_file_lines:
    data = line.split('\t')
    data = [d.strip() for d in data]
    test_lines[data[0]] = data[-1]
    try:
        if fold_dic[data[0]]['PDB-chain'] not in fold_pdb.keys():
            fold_pdb[fold_dic[data[0]]['PDB-chain']] = []
        fold_pdb[fold_dic[data[0]]['PDB-chain']].append(data[-1])
        if fold_dic[data[0]]['PDB-chain'] not in test_pdb.keys():
            test_pdb[fold_dic[data[0]]['PDB-chain']] = []
        test_pdb[fold_dic[data[0]]['PDB-chain']].append(data[-1])
    except:
        failed_test_scop.append(data[0])

In [45]:
with open('../data/HomologyTAPE/training.txt') as f:
    train_file_lines = f.readlines()
train_lines = {}
train_pdb = {}
failed_train_scop = []
for line in train_file_lines:
    data = line.split('\t')
    data = [d.strip() for d in data]
    train_lines[data[0]] = data[-1]
    try:
        if fold_dic[data[0]]['PDB-chain'] not in fold_pdb.keys():
            fold_pdb[fold_dic[data[0]]['PDB-chain']] = []
        fold_pdb[fold_dic[data[0]]['PDB-chain']].append(data[-1])
        if fold_dic[data[0]]['PDB-chain'] not in train_pdb.keys():
            train_pdb[fold_dic[data[0]]['PDB-chain']] = []
        train_pdb[fold_dic[data[0]]['PDB-chain']].append(data[-1])
    except:
        failed_train_scop.append(data[0])

In [46]:
with open('../data/HomologyTAPE/validation.txt') as f:
    val_file_lines = f.readlines()
val_lines = {}
val_pdb = {}
failed_val_scop = []
for line in val_file_lines:
    data = line.split('\t')
    data = [d.strip() for d in data]
    val_lines[data[0]] = data[-1]
    try:
        if fold_dic[data[0]]['PDB-chain'] not in fold_pdb.keys():
            fold_pdb[fold_dic[data[0]]['PDB-chain']] = []
        fold_pdb[fold_dic[data[0]]['PDB-chain']].append(data[-1])
        if fold_dic[data[0]]['PDB-chain'] not in val_pdb.keys():
            val_pdb[fold_dic[data[0]]['PDB-chain']] = []
        val_pdb[fold_dic[data[0]]['PDB-chain']].append(data[-1])
    except:
        failed_val_scop.append(data[0])

In [47]:
fold_dic['d1ysya1']
len(fold_pdb)

10985

In [48]:
for key, value in test_pdb.items():
    test_pdb[key] = list(set(value))
for key, value in train_pdb.items():
    train_pdb[key] = list(set(value))
for key, value in val_pdb.items():
    val_pdb[key] = list(set(value))

In [49]:
print(len(test_file_lines))
print(len(train_file_lines))
print(len(val_file_lines))
print(len(failed_test_scop))
print(len(failed_train_scop))
print(len(failed_val_scop))

718
12312
736
17
550
11


In [50]:
all_pdb = list(test_pdb.keys()) + list(train_pdb.keys()) + list(val_pdb.keys())

In [51]:
all_pdb = [pdb[:4] for pdb in all_pdb]
print(len(all_pdb))

11325


In [57]:
import json
no_download = []
with open('../data/HomologyTAPE/all_download.json') as f:
    download_pdb = json.load(f)
with open('../data/HomologyTAPE/download_pdb.json') as f:
    download_fold = json.load(f)

In [61]:
download_pdb = [i.upper() for i in download_pdb]
download_fold = [i.upper() for i in download_fold]

In [64]:
no_download = []
for pdb in tqdm(all_pdb):
    if pdb not in download_pdb or pdb not in download_fold:
        no_download.append(pdb)

100%|██████████| 11325/11325 [00:36<00:00, 307.38it/s]


In [63]:
len(no_download)

17096

In [54]:
pdb_chains = list(test_pdb.keys()) + list(train_pdb.keys()) + list(val_pdb.keys())

In [55]:
sum = 0
single_pdb_chains = []
for pdb_chain in pdb_chains:
    if pdb_chain in multi_uniprot:
        # print(pdb_chain)
        sum+= 1
    else:
        single_pdb_chains.append(pdb_chain)
print(sum)

36


In [85]:
sum = 0
with open('../data/HomologyTAPE/nrPDB-fold_annot.tsv', 'w') as f:
    for key, value in fold_pdb.items():
        if key not in multi_uniprot:
            f.writelines(key + '\t' + ','.join(value) + '\n')
        else:
            sum += 1
print(sum)
    

36


In [66]:
with open('../data/EnzymeCommission/nrPDB-EC_annot_copy.tsv') as f:
    ec_lines = f.readlines()
with open('../data/GeneOntology/nrPDB-GO_annot.tsv') as f:
    go_lines = f.readlines()
download_chains = []
download_ec_chains = [ec.split('\t')[0] for ec in ec_lines[1:]]
download_go_chains = [ec.split('\t')[0] for ec in go_lines[12:]]
download_chains = download_ec_chains + download_go_chains

In [87]:
import os
download_fold = os.listdir('../data/HomologyTAPE/all/')
download_fold = [i.split('.')[0] for i in download_fold]

In [89]:
download_chains = list(set(download_chains))
need_download = []
already_download = []
for pdb in tqdm(single_pdb_chains):
    if pdb not in download_chains and pdb[:4] not in download_pdb and pdb[:4] not in download_fold:
        need_download.append(pdb)
    else:
        already_download.append(pdb)

  0%|          | 4/11289 [00:00<04:46, 39.37it/s]

100%|██████████| 11289/11289 [01:02<00:00, 180.87it/s]


In [71]:
download_pdb[0]
# pdb[:4]

'8AHS'

In [68]:
print(len(need_download))
print(len(already_download))

9356
1933


In [72]:
already_download = list(set(already_download))
len(already_download)

1840

In [76]:
import os
need_download = list(set(need_download))
# len(need_download)
print(len(os.listdir('../data/HomologyTAPE/all')))
print(len(download_fold))

9273
9273


In [80]:
for i in need_download:
    if i[:4] not in download_fold:
        print(i[:4])

1ATG
1MC2
1A6D
2AR1
1PYB
1Y71
2GF0
1Z24
9PCY
1KPL
1H6Q
1F9N
1OKR
1AX8
1XP8
1NAR
1T3U
1PMP
1KGS
2FE0
1J5Y
1JDL
2G39
2FBQ
2NS1
1P9K
1AG9
1B9R
1I4J
1I6U
7FAB
1TFS
1WXQ
1GKS
1LTR
1HUU
1HQL
7FAB
1SJ1
1VC1
1WRU
1QB5
2D1H
1YB2
1KX7
1JO0
1N1C
1S05
1GMU
1YWQ
1IT2
1WTL
2BI0
1ZBR
1E09
1N4X
1FRD
1FXK
1NR0
3EBX
1D1J
2NLY
1SPP
1AU1
2BYC
1XB4
2GWG
1V6P
2CD0
1JW3
1K3S
1HRO
1VQ0
2PRR
3CDD
1U5K
1UCB
1ID2
1UVQ
1OZ7
1FFK
1M9I
1VPL
2FIY
4BCL
2ALE
3C2C
2SAS
2FB4
1XMX
2PI2
2AIZ
1DZL
1KON
1VKI
1F1F
1F99
1AUA
1GGZ
1P35
2F1L
1JER
1SXJ
1SXJ
1UNA
1VHO
1SXJ
1V4L
1NH5
1RJJ
2FQ4
1LIL
1YKW
2RH7
2JNE
2J5L
1T0K
1NZA
1SFP
1MOZ
2FCR
1SE9
1RH1
1E4F
1IAI
1PVT
1HLQ
1GKA
1PCV
1P4E
1QVR
1XA6
1EQ1
1HH2
2CA6
1G5U
1JS2
1I7H
1OF5
1HEY
2Q4Q
1MQS
1BKB
1AKP
1IMU
1T7V
1ELJ
2FB4
1TBX
1DUW
1Z6X
1EXS
1DLF
1V9J
1G5G
1NTX
1UMR
1IXX
1BF0
1UIS
2GEN
1MNT
1JGK
1E5P
1S3J
1FXK
1IS1
1TEJ
1QSD
1YWX
1A7M
1KR4
1LIT
2PSB
1JF0
2CYY
1GRJ
1LNW
1KN1
1MJC
1ISU
1F6L
1KZQ
1FNN
1QOU
1914
2ARK
1PW4
1R88
1JFI
3BED
2EUI
1N93
1V5W
1VL4
2NRA
1G7S
1BBH
1M5X
1XI8


In [79]:
download_fold[1]

'1W0T'

In [90]:
test_list = already_download[:int(0.2*len(already_download))]
val_list = already_download[int(0.2*len(already_download)):int(0.4*len(already_download))]
train_list = already_download[int(0.4*len(already_download)):]

In [91]:
with open('../data/Fold/test.txt', 'w') as f:
    for i in test_list:
        f.writelines(i + '\n')
with open('../data/Fold/train.txt', 'w') as f:
    for i in train_list:
        f.writelines(i + '\n')
with open('../data/Fold/val.txt', 'w') as f:
    for i in val_list:
        f.writelines(i + '\n')

In [52]:
with open('../data/HomologyTAPE/uniformed_labels.json') as f:
    uniformed = json.load(f)

In [92]:
class_map = {}
with open('../data/HomologyTAPE/class_map.txt') as f:
    lines = f.readlines()
for line in lines:
    # print(line)
    data = line[:-1].split('	')
    # print(data)
    data = [d.strip() for d in data]
    class_map[data[0]] = int(data[1])
print(class_map['a.1'])

0


In [93]:
uniformed_labels = {}
all_list = test_list + val_list + train_list
for i in all_list:
    uniformed_labels[i] = {
        'uniprots': uniprot_dic[i],
        'ec': [-1],
        'go': [-1],
        'reaction':[-1],
        'fold': [[class_map[k] for k in fold_pdb[i]]],
        'ppi': -1,
        'lba': -1
    }

In [94]:
with open('../data/Fold/uniformed_labels.json', 'w') as f:
    json.dump(uniformed_labels, f)

In [98]:
uniprots_all = {}
for i in all_list:
    pdb, chain = i.split('-')
    if i not in uniprots_all.keys():
        uniprots_all[i] = {}
    uniprots_all[i][chain] = uniprot_dic[i][0]

In [99]:
with open('../data/Fold/uniprot_dict_test.json', 'w') as f:
    json.dump(uniprots_all, f)

In [90]:
all_list = []
for key, value in fold_pdb.items():
    if key not in multi_uniprot:
        all_list.append(key)

In [91]:
fold_uniprots = {}
for i in all_list:
    fold_uniprots[i] = uniprot_dic[i]
    # uniprots_all[i][chain] = uniprot_dic[i][0]

In [92]:
with open('../output_info/fold_uniprots.json', 'w') as f:
    json.dump(fold_uniprots, f)

In [94]:
with open('../output_info/uniprot_dict_all_reaction.json') as f:
    uniprots_all = json.load(f)
for i in all_list:
    pdb, chain = i.split('-')
    if i not in uniprots_all.keys():
        uniprots_all[i] = {}
    uniprots_all[i][chain] = uniprot_dic[i][0]
with open('../output_info/uniprot_dict_all_fold.json', 'w') as f:
    json.dump(uniprots_all, f)